<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [2.5 Stochastic Programming](https://ndcbe.github.io/CBE60499/02.05-SP.html) | [Contents](toc.html) | [2.7 Pyomo Homework 2](https://ndcbe.github.io/CBE60499/02.07-Pyomo2.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.06-Pyomo1.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.06-Pyomo1.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

In [1]:
# IMPORT DATA FILES USED BY THIS NOTEBOOK
import os,  requests

file_links = [("data/knapsack_data.csv", "https://ndcbe.github.io/CBE60499/data/knapsack_data.csv")]

# This cell has been added by nbpages. Run this cell to download data files required for this notebook.

for filepath, fileurl in file_links:
    stem, filename = os.path.split(filepath)
    if stem:
        if not os.path.exists(stem):
            os.mkdir(stem)
    if not os.path.isfile(filepath):
        with open(filepath, 'wb') as f:
            response = requests.get(fileurl)
            f.write(response.content)


# 2.6 Pyomo Homework 1

**Due Date:** 2/16/2021

In [2]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_glpk()
    helper.download_data(['knapsack_data.xlsx','knapsack_data.csv'])

In [3]:
## IMPORT LIBRARIES
from pyomo.environ import *
import pandas as pd

Special thanks to the Pyomo team for create these excercises as part of their excellent PyomoFest workshop.

## 2.6.1 Pyomo Fundamentals

### 2.6.1.1 Knapsack example

Solve the knapsack problem given below using GLPK and answer the following questions:

1. Which items are acquired in the optimal solution?

2. What is the value of the selected items?

In [4]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# YOUR SOLUTION HERE -- solved
opt = SolverFactory('glpk')
opt.solve(model)
model.display()




Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    weight_con : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


**Question Answers**

1. A hammer, screwdriver, and a towel are acquired

2. The optimal total value of the selected items is 25

### 2.6.1.2 Knapsack example with improve printing

Complete the missing lines in the code below to produce formatted output: print the total weight, the value of the items selected (the objective), and the items acquired in the optimal solution. Note, the Pyomo value function should be used to get the floating point value of Pyomo modeling components (e.g., print(value(model.x[i])).

In [5]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*value(model.x[i]) for i in A )
# YOUR SOLUTION HERE
A_sel = A.copy()
for i in A:
    if value(model.x[i]) == 0:
        A_sel.remove(i)
        
selstr = "In the optimal solution,"
for i in range(len(A_sel)):
    if i != 0 and len(A_sel)>2:
        selstr += ","
    selstr += " "
    if i == len(A_sel)-1:
        selstr += "and "
    selstr += "a " + A_sel[i]
if len(A_sel) == 0:
    selstr += " nothing"
if len(A_sel) > 1:
    selstr += " are selected."
else:
    selstr += " is selected."
    
print('Total weight = {tweight}'.format(tweight=total_weight))
print('Optimal total value of items selected = {objval}'.format(objval=value(model.obj)))
print(selstr)

print(value(model.x['wrench']))

print('-------------------------')

Total weight = 12.0
Optimal total value of items selected = 25.0
In the optimal solution, a hammer, a screwdriver, and a towel are selected.
0.0
-------------------------


### 2.6.1.3 Changing data

Using your code from **Question 1.2**, if we were to increase the value of the wrench, at what point would it become selected as part of the optimal solution?

In [6]:
# YOUR SOLUTION HERE
while  'wrench' not in A_sel and b['wrench'] < 30:
    b['wrench'] += 1
    model = ConcreteModel()
    model.x = Var( A, within=Binary )
    model.obj = Objective(
        expr = sum( b[i]*model.x[i] for i in A ), 
        sense = maximize )
    model.weight_con = Constraint(
        expr = sum( w[i]*model.x[i] for i in A ) <= W_max )
    opt_success = opt.solve(model)
    total_weight = sum( w[i]*value(model.x[i]) for i in A )
    A_sel = A.copy()
    for i in A:
        if value(model.x[i]) == 0:
            A_sel.remove(i)
print('The wrench would become part of the solution when its value >= {}'.format(b['wrench']))
selstr = "In the optimal solution,"
for i in range(len(A_sel)):
    if i != 0 and len(A_sel)>2:
        selstr += ","
    selstr += " "
    if i == len(A_sel)-1:
        selstr += "and "
    selstr += "a " + A_sel[i]
if len(A_sel) == 0:
    selstr += " nothing"
if len(A_sel) > 1:
    selstr += " are selected."
else:
    selstr += " is selected."
    
print('Total weight = {tweight}'.format(tweight=total_weight))
print('Optimal total value of items selected = {objval}'.format(objval=value(model.obj)))
print(selstr)

print(value(model.x['wrench']))

print('-------------------------')



The wrench would become part of the solution when its value >= 8
Total weight = 14.0
Optimal total value of items selected = 25.0
In the optimal solution, a wrench, a screwdriver, and a towel are selected.
1.0
-------------------------


**Question Answer**

The wrench would become part of the optimal solution when its value is greater than or equal to 8.

### 2.6.1.4 Loading data from Excel

In the code above, the data is hardcoded at the top of the file. Instead of hardcoding the data, use Python to load the data from a difference source. You may use Pandas to load data from 'knapsack_data.xlsx' into a dataframe. You will then need to write code to obtain a dictionary from the dataframe.

In [7]:
df_items = pd.read_excel('./data/knapsack_data.xlsx', sheet_name='data', header=0, index_col=0)
W_max = 14

A = df_items.index.tolist()
# YOUR SOLUTION HERE
b = {}
w = {}
s_benefit = df_items['Benefit']
s_weight = df_items['Weight']
for i in A:
    b[i] = s_benefit[i]
    w[i] = s_weight[i]


model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*value(model.x[i]) for i in A )
print('Total Weight:', total_weight)
print('Total Benefit:', value(model.obj))

print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    acquired = 'No'
    if value(model.x[i]) >= 0.5:
        acquired = 'Yes'
    print('%12s %12s' % (i, acquired))
print('-------------------------')

Total Weight: 12.0
Total Benefit: 25.0
        Item     Selected
      hammer          Yes
      wrench           No
 screwdriver          Yes
       towel          Yes
-------------------------


### 2.6.1.5 NLP vs. MIP

Solve the knapsack problem with IPOPT instead of glpk. Print the solution values for model.x. What happened? Why?

*Hint*: Switch `glpk` to `ipopt` in the call to `SolverFactory`.

In [8]:
from pyomo.environ import *

A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# YOUR SOLUTION HERE
opt = SolverFactory('ipopt')
opt_success = opt.solve(model)

model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'hammer', 'wrench', 'screwdriver', 'towel'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value              : Upper : Fixed : Stale : Domain
             hammer :     0 :                1.0 :     1 : False : False : Binary
        screwdriver :     0 :                1.0 :     1 : False : False : Binary
              towel :     0 :                1.0 :     1 : False : False : Binary
             wrench :     0 : 0.2857142884855869 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight_con : Size=1, Index=None, Active=True
        Key  : Lower : Body                           

**Question Answers**

There is now a fractional number of wrenches

## 2.6.2 More Pyomo Fundamentals

### 2.6.2.1 Knapsack problem with rules

Rules are important for defining in-dexed constraints, however, they can also be used for single (i.e. scalar) constraints. Reimplement the knapsack model from **Question 1.1** using rules for the objective and the constraints.

In [9]:
# YOUR SOLUTION HERE
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

def objrule(model):
    return sum(b[i]*model.x[i] for i in A)

model.obj = Objective(
    rule=objrule, 
    sense = maximize )

def conrule(model):
    return sum( w[i]*model.x[i] for i in A ) <= W_max

model.weight_con = Constraint(rule=conrule)

# YOUR SOLUTION HERE -- solved
opt = SolverFactory('glpk')
opt.solve(model)
model.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    weight_con : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


### 2.6.2.2 Integer formulation of the knapsack problem

Consider again the knapsack problem. Assume now that we can acquire multiple items of the same type. In this new formulation, $x_i$ is now an integer variable instead of a binary variable. One way to formulate this problem is as follows:


$\max_{q,x} \sum_{i\in{A}}v_ix_i$

s.t. $\sum_{i\in{A}}w_ix_i\leq W_{max}$

$x_i=\sum_{j=0}^Njq_{i,j}, \forall i \in{A}$

$0\leq x\leq N$

$q_{i,j} \in {0,1}, \forall i \in A, j \in {0...N}$

Starting with your code from **Question 2.1**, implement this new formulation and solve. Is the solution surprising?

In [10]:
# YOUR SOLUTION HERE
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=NonNegativeIntegers )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# YOUR SOLUTION HERE -- solved
opt = SolverFactory('glpk')
opt.solve(model)
model.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        screwdriver :     0 :   0.0 :  None : False : False : NonNegativeIntegers
              towel :     0 :   4.0 :  None : False : False : NonNegativeIntegers
             wrench :     0 :   0.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  44.0

  Constraints:
    weight_con : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


**Question Answer**

The solution is unsurprising because towel has the greatest benefit/weight ratio, so of course the optimizer would add as many towels as possible

<!--NAVIGATION-->
< [2.5 Stochastic Programming](https://ndcbe.github.io/CBE60499/02.05-SP.html) | [Contents](toc.html) | [2.7 Pyomo Homework 2](https://ndcbe.github.io/CBE60499/02.07-Pyomo2.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.06-Pyomo1.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.06-Pyomo1.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>